## 各种import

In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("..\examples"))

import utils; reload(utils)
from utils import *

from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop

Using Theano backend.


## 从Keras导入数据

In [2]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000L, 28L, 28L), (60000L,), (10000L, 28L, 28L), (10000L,))

## 处理数据

### One-Hot-Encoding

In [3]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道

In [4]:
trn_data = np.expand_dims(trn_data, 1)
test_data = np.expand_dims(test_data, 1)
trn_data.shape

(60000L, 1L, 28L, 28L)

### 对输入图像进行正则化

In [6]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

## 1. 简单神经网络模型

In [14]:
linear_model = Sequential()
linear_model.add(Lambda(norm_input, input_shape=(1, 28, 28)))
linear_model.add(Flatten())
linear_model.add(Dense(392, activation='softmax'))
linear_model.add(Dense(10, activation='softmax'))
linear_model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

C:\anaconda2\lib\site-packages\keras\layers\core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


### 1.1 直接训练

In [8]:
linear_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 11s - loss: 1.2822 - acc: 0.8498 - val_loss: 0.6254 - val_acc: 0.8979


### 1.2 生成batch再训练(利用ImageDataGenerator())